## Import TensorFlow and other libraries

In [ ]:
import gzip
import numpy as np
import os
import pathlib

assert "SDI/codespace/dataset_training" in os.getcwd().replace("\\", "/")
if not os.getcwd().endswith("dataset_training"):
    os.chdir("../..")
os.getcwd()

In [ ]:
from GAF.preprocessing.features_extractors.raw_extractor import RawExtractor
from GAF.preprocessing.data_translator.gaf_translator import GafTranslator
from GAF.preprocessing.preprocessor import Preprocessor

TIME_PER_SAMPLE = 2
WINDOWS = 4
RESAMPLE_MS = TIME_PER_SAMPLE * 1000 // WINDOWS
SAMPLES_PER_MINUTE = 60
PATH = f"data/processed/fixed_{SAMPLES_PER_MINUTE}"
ANOMALIES_PATH = f"anomalies/processed/fixed_{SAMPLES_PER_MINUTE}"
AXIS_WINDOWS_AMOUNT = (
    TIME_PER_SAMPLE * SAMPLES_PER_MINUTE
)  # 4 windows * 0.500 s/window * 60 points-in-sample/s = 120 points-in-sample
AXES = ["x", "y", "z", "tot"]

extractor = RawExtractor(resample_amount=RESAMPLE_MS)
preprocess = Preprocessor(extractor, packed_windows=WINDOWS, path=PATH)
anomalies_preprocess = Preprocessor(extractor, packed_windows=WINDOWS, path=ANOMALIES_PATH)

In [ ]:
sensor = "gyroscope"


class SmallGafDatasetLoader:
    def __init__(self, preprocess, sensor):
        self.data, self.labels, self.ids = preprocess.load_dataset_sensor(sensor)

In [ ]:
train_dataset = SmallGafDatasetLoader(preprocess, sensor)
anomaly_dataset = SmallGafDatasetLoader(anomalies_preprocess, sensor)

# To get the name used by the program we will use this
preprocess_real = Preprocessor(
    extractor,
    translators=[GafTranslator(AXIS_WINDOWS_AMOUNT, WINDOWS)],
    packed_windows=WINDOWS,
    path=PATH,
)
anomalies_preprocess_real = Preprocessor(
    extractor,
    translators=[GafTranslator(AXIS_WINDOWS_AMOUNT, WINDOWS)],
    packed_windows=WINDOWS,
    path=ANOMALIES_PATH,
)

In [ ]:
print(np.array(train_dataset.ids).shape)
print(np.array(train_dataset.data).shape)
print(np.array(anomaly_dataset.ids).shape)
print(np.array(anomaly_dataset.data).shape)

In [ ]:
translator = GafTranslator(AXIS_WINDOWS_AMOUNT, WINDOWS, method="summation")


# translator = GafTranslator(AXIS_WINDOWS_AMOUNT, WINDOWS, method='difference')
def save_info(dataset, output_dir_name):
    print(np.array(dataset.ids).shape)
    print(np.array(dataset.data).shape)
    for axis in AXES:
        i = 0
        for gaf_image in translator.translate_axis_yield(dataset.data, axis):
            path = pathlib.Path(f"{output_dir_name}/{axis}/{i}.npy")
            path.parent.mkdir(parents=True, exist_ok=True)
            # print(path, end="\r")
            f = gzip.GzipFile(path, "w")
            np.save(f, gaf_image)
            i += 1

            # loading example
            # f = gzip.GzipFile(path, "r")
            # m = np.load(f)
            # print(m.shape)


save_info(train_dataset, preprocess_real.get_file_name(sensor, [translator]))
save_info(anomaly_dataset, anomalies_preprocess_real.get_file_name(sensor, [translator]))